## Topographische und hydrographische Modelle (120.109)

### Protokoll 3

### Präambel

Fügen Sie Ihren Code bzw. ihre Antworten an den Stellen ein, die mit `YOUR CODE HERE` oder \"YOUR ANSWER HERE\" gekennzeichnet sind. **Wichtig**: Bevor Sie die Aufgabe abgeben, stellen Sie sicher, dass die `raise NotImplementedError()` Zeilen aus Ihrem Code entfernt wurden, und dass Ihr Code fehlerlos durchläuft. Starten Sie dazu den Kernel neu und führen Sie alle Zellen aus (Doppelpfeilsymbol in der Menüleiste). Löschen Sie vor der Abgabe alle Zwischen- und Ergebnisdateien außer die in QGIS erstellten PDF-Karten (siehe Hinweis bei Aufgabe 3).

#### Aufgabe 1: Laserbathymetrie (10 Punkte)
Gegeben ist die unkorrigierte 3D Punktwolke (*laserbathy.laz*) einer topo-bathymetrischen UAV-Laserscanning Befliegung eines Bereiches der Pielach zwischen Loosdorf und Melk (NÖ), die zugehörige Flugtrajektorie (*trajectory.txt*) samt Formatbeschreibungsdatei (*trajectory.xml*) sowie die mittlere Wasserspeigelhöhe im Untersuchungsgebiet von 260.10 m. Die Dateien befinden sich im Verzeichnis *~/shared/Daten/Protokoll_3*. Die Punktwolke ist bereits klassifiziert (gespeichert in den Attributen *Classification* und *UserData*) gemäß folgender Klasseneinteilung:

| ID | Beschreibung |
|:---|:-------------|
| 0  | nicht klassifiziert  |
| 2  | Bodenpunkt (über und unter Wasser)  |
| 3  | niedrige Vegetation |
| 4  | mittlere Vegetation |
| 5  | hohe Vegetation |
| 9  | Wasserkörper |

Sowohl die Punktwolke (x, y, z + Attribute) als auch die Trajektorie (x, y, z, roll, pitch, yaw) ist zeitgestempelt, sodass für jeden Laserpuls die zugehörige Strahlrichtung (beam vector) berechnet werden kann. Diese bildet zusammen mit dem Wasseroberflächenmodell (konkret: horizontale Fläche auf Höhe 260.10) die Grundlage für die Refraktionskorrektur. Einige Kenngrößen der Datenerfassung

- Flughöhe: 50 m über Grund
- Scanpattern: elliptisch (Palmer Scanner, $\pm14^{\circ}$ vorwärts/rückwärts, $\pm20^{\circ}$ links/rechts)
- Pulswiederholrate: 200 kHz
- Strahlöffnungswinkel: 2 mrad
- Footprintdurchesser am Boden: 10 cm

![](pcl-laserbathy.png)

Berechnen Sie ein **Wasserlauf-Geländemodell (DGM-W)** und ein **Wassertiefenmodell** und visualisieren Sie die Modelle gemäß den folgenden Angaben:

- Bestimmen Sie für jeden Punkt der 3D Punktwolke die Laserstrahlrichtung (beam vector) durch gemeinsamen  [Import](https://opals.geo.tuwien.ac.at/html/stable/ModuleImport.html) von Punktwolke und Trajektorie. Speichern Sie die um die Strahlrichtung erweiterte Punktwolke in der Datei *laserbathy.odm*.
- Führen Sie die Laufzeit- und Refraktionskorrektur gemäß dem  [Snellius](https://opals.geo.tuwien.ac.at/html/stable/ModuleSnellius.html)'schen Brechnungsgesetz durch. Verwenden Sie für die Korrektur einen Brechungsindex von $n_w=1.35$. Speichern Sie die brechungskorrigierte Punktwolke in der Datei *laserbathy_refCorr.odm*. Anmerkung: Da das Modul [Snellius](https://opals.geo.tuwien.ac.at/html/stable/ModuleSnellius.html) die Klassifizierung der Unterwasserpunkte verändert, stellen Sie danach die ursprüngliche Klassifizierung aus dem Feld *UserData* wieder her. Verwenden Sie dazu das Modul [AddInfo](https://opals.geo.tuwien.ac.at/html/stable/ModuleAddInfo.html)
- Berechnen Sie aus der refraktionskorrigierten Punktwolke ein DGM-W mit dem Modul [Grid](https://opals.geo.tuwien.ac.at/html/stable/ModuleGrid.html) und speichern Sie dieses als *laserbathy_dgmw.tif*
  - Verwenden Sie dazu die Bodenpunkte 
  - Interpolationsmethode: moving planes
  - Gitterweite: 10cm
  - Nachbarnanzahl: 16
  - maximaler Suchradius: 2m
  - quadrantenweise Nachbarnsuche
- Visualisieren Sie das DGM-W mittels Schummerung und Z-Coding und speichern Sie diese unter *laserbathy_dgmw_shd.tif* und *laserbathy_dgmw_zco.tif*. Verwenden Sie dazu geeignete Parameter ihrer Wahl.
- Berechnen Sie aus dem DGM-W und der Wasserspiegelhöhe mit dem Modul [Algebra](https://opals.geo.tuwien.ac.at/html/stable/ModuleAlgebra.html) ein Wassertiefenmodell und speichern Sie dieses als *laserbathy_wtiefe.tif* (Wassertiefe positiv!).
- Visualisieren Sie das Wassertiefenmodell als farbkodierte Rasterkarte und in Form von Höhenlinien.
  - Farbkodierte Wassertiefen-Rasterkarte:
    - Äquidistanz: 10cm
    - Palette: colorBrewer_SpectralPal.xml
    - Dateiname: *laserbathy_wtiefe_zco.tif*
  - Höhenlinien:
    - Äquidistanz: 10cm
    - Geeignete Parameter für minimale Fläche und Länge
    - Dateiname: *laserbathy_wtiefe_iso.shp*

In [1]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [2]:
###################################################################
# Import Statements
###################################################################
import rasterio
import shapely
import fiona
import os

###################################################################
# Pfad zum Datenverzeichnis
###################################################################
datadir = os.path.join( os.getenv('HOME'), 'shared', 'Daten', 'Protokoll_3')

In [3]:
###################################################################
# Sicherstellung der Nicht-Existenz der Ergbnisdateien
###################################################################
!rm -f *.odm
!rm -f *.svg
!rm -f *.log
!rm -f laserbathy_wtiefe*
!rm -f laserbathy_dgmw*
!rm -f opalsLog.xml

In [4]:
# YOUR CODE HERE

#Modulimport
import opals
from opals import Import, Snellius, AddInfo, Grid, Shade, ZColor, Algebra, Isolines, Translate
from os.path import join


#Dateinamen
dataset = 'laserbathy'
lazF = join(datadir, dataset + '.laz')
odmF = dataset + '.odm'
odmF_corr = dataset + '_refCorr.odm'

trjF = join(datadir, 'trajectory.txt')
trfF = join(datadir, 'trajectory.xml')

dgmF = dataset + '_dgmw.tif'
wdeF = dataset + '_wdepth.tif'
wlev = 260.10


#File Importieren
Import.Import(inFile = lazF, outFile = odmF, trjFile =trjF, tFormat  = trfF, storeBeamInfo = [opals.Types.BeamInfo.BeamVector]).run()

#Refraktionskorrektur
refI = 1.35
Snellius.Snellius(inFile=odmF,  outFile=odmF_corr, refModel=str(wlev), refractiveIndex=refI).run()

# Klassifizierungswiederherstellung
AddInfo.AddInfo(inFile = odmF_corr, attribute = 'Classification=UserData')

#DGM-W
Grid.Grid(inFile = odmF_corr, outFile = dgmF, interpolation = opals.Types.GridInterpolator.movingAverage, gridSize = 0.1, neighbours = 16, searchRadius = 2, selMode = opals.Types.SelectionMode.quadrant).run() #filter = 'class[Ground]'

#Visualisierung DGMW
palF = 'standardPal.xml'
scaleP = '256 270'
Shade.Shade(inFile = dgmF, outFile = 'laserbathy_dgmw_shd.tif', sunPosition=[350,50]).run()
ZColor.ZColor(inFile = dgmF, outFile = 'laserbathy_dgmw_zco.tif', interval = 0.5, palFile = palF, scalePal=scaleP).run()

#Wassertiefenmodell
Algebra.Algebra(inFile = dgmF, formula = 'r[0] < 260.10 ? 260.10 - r[0] : invalid', outFile = wdeF).run()

#Visualisierung Wassertiefenmodell
palF = 'colorBrewer_SpectralPal.xml'
scaleP = '0 1.5'
ZColor.ZColor(inFile=wdeF, outFile='laserbathy_wtiefe_zco.tif', palFile=palF, scalePal=scaleP, interval=0.1).run()

Isolines.Isolines(inFile=wdeF, outFile='laserbathy_wtiefe_iso.shp', interval=0.1, minLength = 2, closeMin = 3).run()

[Parameter(path=inFile, values=laserbathy_wdepth.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=laserbathy_wtiefe_iso.shp, type=Path, source=interFace),
 Parameter(path=oFormat, values=shape, type=String, source=estimated),
 Parameter(path=band, values=0, type=String, source=internalDefault),
 Parameter(path=interval, values=0.10000000000000001, type=double, source=interFace),
 Parameter(path=levels, values=, type=Vector<double>, source=unknown),
 Parameter(path=zRange, values=, type=Array<double,2>, source=unknown),
 Parameter(path=thinOut, values=0.01, type=double, source=internalDefault),
 Parameter(path=closeMin, values=3, type=double, source=interFace),
 Parameter(path=minLength, values=2, type=double, source=interFace),
 Parameter(path=densification, values=1, type=bool, source=internalDefault)]

In [5]:
# Sichtbare Überprüfungen
assert os.path.exists('laserbathy.odm')
#
assert os.path.exists('laserbathy_refCorr.odm')
#
assert 'Import' in globals()
assert 'Snellius' in globals()
assert 'AddInfo' in globals()
assert 'Grid' in globals()
assert 'Shade' in globals()
assert 'ZColor' in globals()
assert 'Algebra' in globals()
assert 'Isolines' in globals()
assert 'Translate' in globals()

In [6]:
# Verborgene Überprüfungen 1 (DGM-W, photobathy_wtiefe_zco.tif)

In [7]:
# Verborgene Überprüfungen 2 (laserbathy_dgmw_zco.tif)

In [8]:
# Verborgene Überprüfungen 3 (laserbathy_dgmw_shd.tif)

In [9]:
# Verborgene Überprüfungen 4 (laserbathy_wtiefe.tif)

In [10]:
# Verborgene Überprüfungen 5 (laserbathy_wtiefe.tif.shp, laserbathy_wtiefe_zco.tif)

In [11]:
# Verborgene Überprüfungen 6 (Refraktionskorrektur Datei laserbathy.odm, Min/Max)

In [12]:
# Verborgene Überprüfungen 7 (Refraktionskorrektur Datei laserbathy.odm, Mean/Median/StdDev)

### Aufgabe 1b

Beschreiben Sie in ca 100-200 Worten kurz das Grundkonzept der Laserbathymetrie mit Fokus auf Refraktionskorrektur. Verwenden Sie dazu das gelernte Wissen aus den Vorlesungseinheiten und die entsprechenden Unterlagen aus dem Skriptum zur Lehrveranstaltung. Gehen Sie dabei auf die wesentlichen Punkte Wasseroberfläche, Strahlablenkung, Ausbreitungsgeschwindigkeit in Luft und Wasser sowie auf das physikalische Grundgesetz ein. 

Die Laserbathymetrie ist eine Methode, um die topografischen Eigenschaften von Gewässern zu untersuchen. Sie ist eine aktive Methode und funktioniert mittels Laufzeitmessung, es wird also ein Laserstrahl ausgesandt und die Zeit gemessen bis zu seiner Rückkehr, um daraus über die Lichtgeschwindigkeit die Höhe des untersuchten Punktes zu bestimmen. Die Vorteile der Laserbathymetrie sind die große Eindringtiefe im Vergleich zu passiven Methoden sowie die Höhengenauigkeit. Meist wird Licht mit grüner Wellenlänge verwendet, da dieses vom Wasser nicht so stark absorbiert wird wie andere Wellenlängen.

Das Prinzip der Laufzeitmessung birgt jedoch auch Probleme. So ist die Lichtgeschwindigkeit, mit der sich der Laserstrahl ausbreitet, im Wasser um ca. ein Drittel geringer als in der Luft. Das führt unter anderem dazu, dass das Licht an der Wasseroberfläche gebrochen wird. Dieser Umstand macht die Berechnung der geometrischen Ausbreitung des Laserstrahls, die für die korrekte Ermittlung der Geländehöhe notwendig ist, schwierig. Es muss der Brechungsvorgang in die Formeln miteinbezogen werden, um keine falschen (meist zu flachen) Wassertiefen zu erhalten. Das funktioniert über das Snellius’sche Brechungsgesetz, das das Verhältnis vom Sinus des Einfallswinkels und dem Sinus des Aufallswinkels dem umgekehrten Verhältnis der Brechungsindizes von Luft und Wasser gleichsetzt.